In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import *
from scipy.stats import skew
from tqdm import tqdm
from sklearn.model_selection import *
from sklearn.decomposition import PCA
from sklearn.ensemble import *
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

In [ ]:
train_df=pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv')
test_df=pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')
sample_sub=pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv')

In [ ]:
train_df.head(10)

In [ ]:
print('Dataframe shape: ',train_df.shape)

In [ ]:
train_df.describe()

In [ ]:
#Drop Unnecessary feature 'id'
train_df.drop('id',axis=1,inplace=True)

#Get column names
columns=train_df.drop('target',axis=1).columns
print('Number of columns: ',len(columns))

**Plotting**
>Distribution plot

In [ ]:
row,cols=4,4
fig=plt.figure(figsize=(18,18))
for i in range(1,row*cols+1):
    if i>14:
        break
    fig.add_subplot(row,cols,i)
    plt.hist(train_df[columns[i-1]],bins=55)
    plt.title(columns[i-1])

>Plot to check outliers



In [ ]:
row,cols=4,4
fig=plt.figure(figsize=(18,18))
for i in range(1,row*cols+1):
    if i>14:
        break
    fig.add_subplot(row,cols,i)
    plt.boxplot(train_df[columns[i-1]])
    plt.title(columns[i-1])

In [ ]:
sns.distplot(np.sqrt(train_df['target']))

>Correlation Matrix

In [ ]:
plt.figure(figsize=(11,11))
corrMat=train_df.corr()
sns.heatmap(corrMat,linewidths=0.8,annot=True)
plt.show()

In [ ]:
ss=StandardScaler()


In [ ]:
train_df,val_df=train_test_split(train_df,test_size=0.2)
X_train,y_train=train_df.drop('target',axis=1),train_df['target'].values
X_val,y_val=val_df.drop('target',axis=1),val_df['target'].values
print('Training Shape: ',train_df.shape)
print('Validation Shape: ',val_df.shape)

In [ ]:
X_train=ss.fit_transform(X_train)
X_val=ss.transform(X_val)

In [ ]:
#Linear Regression
lgb=LGBMRegressor(learning_rate=0.1,num_leaves=250,max_depth=10,
                  min_child_samples=500)
                 
lgb.fit(X_train,y_train)
print('Validation RMSE: ')
np.sqrt(mean_squared_error(y_val,lgb.predict(X_val)))

In [ ]:
#Kfold on complete dataset

train_df=pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv')
X_train,y_train=train_df.drop(['id','target'],axis=1).values,train_df['target'].values
y_train=y_train

kf=KFold(n_splits=5)
fold=0
for train_ix,test_ix in kf.split(X_train,y_train):
    fold+=1
    x_train_,y_train_=X_train[train_ix],y_train[train_ix]
    x_val_,y_val_=X_train[test_ix],y_train[test_ix]  
    
    x_train_=ss.fit_transform(x_train_)
    x_val_=ss.transform(x_val_)
    
    lgb.fit(x_train_,y_train_)
    train_score=np.sqrt(mean_squared_error(y_train_,lgb.predict(x_train_)))
    val_score=np.sqrt(mean_squared_error(y_val_,lgb.predict(x_val_)))
    print('Fold {} Train Score: {:.3f} Val Score: {:.3f}'.format(fold,train_score,val_score))

In [ ]:
#Test Data
test_data=ss.transform(test_df.drop('id',axis=1))
predictions=lgb.predict(test_data)

sample_sub['target']=predictions
sample_sub.to_csv('submission.csv',index=False)